In [18]:
from transformers import LongformerTokenizerFast
from datasets import load_dataset
import ftfy
from tqdm.auto import tqdm

In [16]:
tokenizer = LongformerTokenizerFast.from_pretrained("allenai/longformer-base-4096")

In [17]:
dataset = load_dataset("wikipedia", language="pt", date="20230401",  beam_runner='DirectRunner')
wiki = dataset.remove_columns([col for col in dataset.column_names if col != "text"])
# dataset = load_dataset("imdb", split="train")

Found cached dataset imdb (/Users/israelcampiotti/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0)


In [19]:
# def batch_iterator(n_samples=None):
#     for i, example in enumerate(dataset["train"]):
#         if n_samples is not None and i > n_samples:
#             break
#         text = example["title"] + "\n" + example["text"]
#         text = ftfy.fix_text(text)
#         yield text
def batch_iterator(batch_size=100):
    for i in tqdm(range(0, len(dataset), batch_size)):
        text = dataset[i : i + batch_size]["text"]
        text = [ftfy.fix_text(t) for t in text]
        yield text

In [20]:
new_tokenizer = tokenizer.train_new_from_iterator(batch_iterator(), vocab_size=tokenizer.vocab_size)

100%|██████████| 250/250 [00:01<00:00, 146.87it/s]


In [21]:
new_tokenizer.save_pretrained("longformer-pt-tokenizer")

('longformer-pt-tokenizer/tokenizer_config.json',
 'longformer-pt-tokenizer/special_tokens_map.json',
 'longformer-pt-tokenizer/vocab.json',
 'longformer-pt-tokenizer/merges.txt',
 'longformer-pt-tokenizer/added_tokens.json',
 'longformer-pt-tokenizer/tokenizer.json')

In [22]:
new_tokenizer.vocab_size

1000